In [1]:
import pandas as pd
import numpy as np
import csv
import os
import re
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier , LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection, svm
from sklearn.svm import SVC as svc
import string
import re 
import nltk
import numpy
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import model_selection, svm
from sklearn.svm import SVC as svc
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import plot_confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
def write_preds(input, inpid, preds,fname, delim=','):
    with open (fname, 'w',encoding='utf8') as f_w:
        writer = csv.writer(f_w, delimiter=delim)
        writer.writerow(['ID', 'stance'])
        for id, inpid, label in zip(input, inpid, preds):
            writer.writerow([inpid,label])

In [3]:
Train_Data = 'Mawqif_AllTargets_Train.csv'
Test_Data  = 'Mawqif_AllTargets_Blind Test.csv'

In [4]:
df = pd.read_csv(Train_Data)
df_test = pd.read_csv(Test_Data)

In [5]:
df = df[['ID','text','stance']]
df_test=df_test[['ID','text']]
mapping = {'None': 0, 'Favor': 1, 'Against': 2}
df['stance'] = df['stance'].apply(lambda x: mapping[x])

In [6]:
test_text=df_test['text']
test_id=df_test['ID']

In [7]:
def process(text):
    text = text.strip()
    text = re.sub(r'(.)\1+', r'\1', text)
    text = re.sub(r'http\S+', ' ', str(text))
    text = re.sub(r'@\w+', ' ', str(text))
    text = re.sub(r'_',' ',text)
    text = re.sub(r'[a-zA-Z0-9]*PM','',text)
#normalis arabic
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [8]:
dt_text=[process(x) for x in df['text']]
dt_test=[process(x) for x in df_test['text']]
dt_label=df['stance']

In [9]:
CV, MAX_FEATURE, MAX_DF, nGram  = 5, None, 1.0, 2
TfIdfVect = TfidfVectorizer(max_df = MAX_DF,
                            max_features = MAX_FEATURE,
                            ngram_range = (1, nGram))

In [10]:
train = TfIdfVect.fit_transform(dt_text)
test =TfIdfVect.transform(dt_test)

In [11]:
clf1  = svm.SVC(kernel = 'linear', random_state=15)

In [12]:
model1 = clf1.fit(train, dt_label)
predicted1 = model1.predict(test)

In [13]:
clf3  = SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, max_iter=10000,random_state=45)

In [14]:
model3 = clf3.fit(train, dt_label)
predicted3 = model3.predict(test)

In [15]:
mapping = {0: 'None', 1: 'Favor', 2: 'Against'}
predicted3_labels = np.vectorize(lambda x: mapping[x])(predicted3)
predicted1_labels = np.vectorize(lambda x: mapping[x])(predicted1)

In [16]:
Result_pred =write_preds(test_text,test_id,predicted1_labels,'Mawqif_AllTargets_Blind Test_1.csv')

In [17]:
Result_pred =write_preds(test_text,test_id,predicted3_labels,'Mawqif_AllTargets_Blind Test_2.csv')